# Fashion MNIST

## Validate Azure ML SDK installation and get version number for debugging purposes

In [ ]:
# Check core SDK version number
import azureml.core
print("SDK version:", azureml.core.VERSION)

## Initialize Workspace
Initialize a workspace object from persisted configuration.

In [ ]:
# Initialize Workspace
from azureml.core import Workspace

ws = Workspace.from_config()
print("Resource group: ", ws.resource_group)
print("Location: ", ws.location)
print("Workspace name: ", ws.name)

## Create a project directory
Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script, and any additional files your training script depends on.

In [ ]:
import os
import shutil

project_folder = '../projects/fashion_mnist_aci'
os.makedirs(project_folder, exist_ok=True)

config_folder = os.path.join(project_folder, 'aml_config')
os.makedirs(config_folder, exist_ok=True)

shutil.copy('./scripts/train_Fashion_MNIST.py', project_folder)
shutil.copy('./resources/configs/fashion_mnist_aci_dependencies.yml', config_folder)
shutil.copy('./resources/configs/fashion_mnist_aci.runconfig', config_folder)

## Create An Experiment
**Experiment** is a logical container in an Azure ML Workspace. It hosts run records which can include run metrics and output artifacts from your experiments.

In [ ]:
from azureml.core import Experiment
experiment_name = 'fashion-mnist'
experiment = Experiment(workspace = ws, name = experiment_name)

## Load Configuration

In [ ]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies

# create a new runconfig object
run_config = RunConfiguration.load(path=project_folder, name='fashion_mnist_aci')

## Submit the Experiment
Finally, run the training job on the ACI

In [ ]:
from azureml.core.script_run_config import ScriptRunConfig

script_run_config = ScriptRunConfig(source_directory=project_folder,
                                    script='train_Fashion_MNIST.py',
                                    run_config=run_config)

run = experiment.submit(script_run_config)
run.tag("Description","ACI trained Fashion MNIST model")

In [ ]:
run

In [ ]:
run.wait_for_completion(show_output=True)

## Show Metrics

In [ ]:
# get all metris logged in the run
run.get_metrics()
metrics = run.get_metrics()

import numpy as np
print('loss is {0:.2f}, and accuracy is {1:0.2f}'.format(
    metrics['loss'], 
    metrics['accuracy']
))

In [ ]:
# Plot data to see relationships in training and validation data
import numpy as np
import matplotlib.pyplot as plt
epoch_list = list(range(1, len(metrics['Training Accuracy']) + 1))  # values for x axis [1, 2, ..., # of epochs]
plt.plot(epoch_list, metrics['Training Accuracy'], epoch_list, metrics['Validation Accuracy'])
plt.legend(('Training Accuracy', 'Validation Accuracy'))
plt.show()

## Download Model

In [ ]:
# show all the files stored within the run record
run.get_file_names()

In [ ]:
import os

outputs_path = os.path.join(project_folder, "outputs")
os.makedirs(outputs_path, exist_ok=True)

for filename in run.get_file_names():
    if filename.startswith('outputs'):
        path = os.path.join(project_folder, filename)
        print("Downloading " + filename)
        run.download_file(filename, output_file_path=outputs_path)